In [19]:
%pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
# Script de génération de données de test pour la base de données LBDR
import mysql.connector
import random
import uuid
from datetime import datetime, timedelta, date
import json

# Configuration de la connexion à la base de données
config = {
    'user': 'user',
    'password': 'userpassword',
    'host': 'localhost',
    'database': 'lbdr_db',
    'port': 3306
}

# Définition d'un dictionnaire pour associer les chemins d'images aux jeux correspondants
images_jeux = {
    "Warhammer": "https://cdn1.epicgames.com/offer/f640a0c1648147fea7e81565b45a3003/EGS_Warhammer40000SpaceMarine2_SaberInteractive_S1_2560x1440-975214651d1d1bc6c6e5779b53958840",
    "L'Appel de Cthulhu": "/data/images/creature-cthulhu-grand-ancien.webp",
    "Donjons et Dragons": "/data/images/donjon&dragon.jpg",
    "Vampire: La Mascarade": "/data/images/vampire_mascarde.avif",
    "Chroniques Oubliées": "/data/images/Chroniques_Oubliees.jpg",
    "Shadowrun": "/data/images/Shadowrun",
    "Numenera": "/data/images/Numenera.jpg",
    "Blades in the Dark": "/data/images/Blades in the Dark.jpg",
    "Cyberpunk RED": "/data/images/Cyberpunk RED.png",
    "Pathfinder": "/data/images/Pathfinder.jpg"
}

# Dictionnaire pour les icônes des jeux
icones_jeux = {
    "Warhammer": "/data/images/icon/Warhammer40k.png",
    "Donjons et Dragons": "/data/images/icon/dnd_logo_big.webp"
    # Les autres jeux n'ont pas d'icônes ou elles sont commentées dans le fichier games.tsx
}

try:
    # Connexion à la base de données
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("Connexion à la base de données réussie!")
    
        # Vider toutes les tables de la base de données dans l'ordre des dépendances
    tables_a_vider = [
        # D'abord les tables avec des clés étrangères
        "refresh_tokens",
        "paiements_helloasso",
        "notifications_helloasso",
        "creneaux_utilisateur",
        "horaires_lieu",
        "joueurs_session",
        "membres_partie",
        "sessions",
        "jeux_genres",
        "evenements",
        "parties",
        # Puis les tables principales
        "lieux",
        "genres",
        "jeux",
        "periodes_association",
        "utilisateurs"
    ]

    # Désactiver temporairement les contraintes de clés étrangères pour faciliter le nettoyage
    cursor.execute("SET FOREIGN_KEY_CHECKS=0")
    
    for table in tables_a_vider:
        try:
            cursor.execute(f"DELETE FROM {table}")
            print(f"✓ Table {table} vidée")
        except mysql.connector.Error as err:
            print(f"! Erreur lors du nettoyage de la table {table}: {err}")
    
    # Réactiver les contraintes de clés étrangères
    cursor.execute("SET FOREIGN_KEY_CHECKS=1")

    conn.commit()
    print("✅ Toutes les tables ont été vidées avec succès!")
    
    # --- FONCTION UTILITAIRES ---
    def generate_password_hash():
        """Génère un hash de mot de passe fictif"""
        return ''.join(random.choice('0123456789abcdef') for _ in range(64))
    
    def random_date(start_date, end_date):
        """Génère une date aléatoire entre start_date et end_date"""
        time_between_dates = end_date - start_date
        days_between_dates = time_between_dates.days
        random_number_of_days = random.randrange(days_between_dates)
        return start_date + timedelta(days=random_number_of_days)
    
    # --- GÉNÉRATION DES UTILISATEURS ---
    print("Génération des utilisateurs...")
    
    # Liste de prénoms et noms pour la génération
    prenoms = ['Alice', 'Bob', 'Charlie', 'David', 'Emma', 'François', 'Gilles', 'Hélène', 
               'Isabelle', 'Jean', 'Kevin', 'Lucie', 'Marc', 'Nathalie', 'Olivier', 'Patricia']
    noms = ['Martin', 'Dupont', 'Durand', 'Lefebvre', 'Moreau', 'Simon', 'Laurent', 
            'Michel', 'Leroy', 'Garcia', 'Bernard', 'Thomas', 'Robert', 'Richard']
    sexes = ['M', 'F', 'Autre']
    types_utilisateur = ['INSCRIT', 'INSCRIT', 'INSCRIT', 'ADMINISTRATEUR']  # Plus de chance d'avoir des inscrits

    # Génération d'utilisateurs
    users_data = []
    for i in range(20):
        user_id = str(uuid.uuid4())
        prenom = random.choice(prenoms)
        nom = random.choice(noms)
        login = f"{prenom.lower()}.{nom.lower()}{random.randint(1, 99)}"
        date_naissance = random_date(date(1970, 1, 1), date(2005, 12, 31))
        sexe = random.choice(sexes)
        id_discord = f"{prenom.lower()}#{random.randint(1000, 9999)}" if random.random() > 0.3 else None
        pseudonyme = f"{prenom.lower()}{random.randint(1, 99)}" if random.random() > 0.3 else None
        email = f"{login}@example.com" if random.random() > 0.2 else None
        mot_de_passe = generate_password_hash()
        image = None  # Pour simplifier
        type_utilisateur = random.choice(types_utilisateur)
        date_inscription = random_date(date(2020, 1, 1), date(2025, 5, 1))
        ancien_utilisateur = random.random() > 0.7
        premiere_connexion = random.random() > 0.8
        
        users_data.append((
            user_id, prenom, nom, login, date_naissance, sexe, id_discord, pseudonyme,
            email, mot_de_passe, image, type_utilisateur, date_inscription,
            ancien_utilisateur, premiere_connexion
        ))
    
    # Insertion des utilisateurs dans la base de données
    user_sql = """
    INSERT INTO utilisateurs (
        id, prenom, nom, login, date_de_naissance, sexe, id_discord, pseudonyme,
        email, mot_de_passe, image, type_utilisateur, date_inscription,
        ancien_utilisateur, premiere_connexion
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(user_sql, users_data)
    conn.commit()
    print(f"✅ {len(users_data)} utilisateurs créés avec succès!")
    

    # --- GÉNÉRATION DES JEUX ---
    print("Génération des jeux...")
    
    jeux_base = [
        # Jeux de rôle
        ("Donjons et Dragons", "Le jeu de rôle fantasy le plus populaire au monde", "JDR"),
        ("L'Appel de Cthulhu", "Jeu d'horreur lovecraftienne", "JDR"),
        ("Warhammer Fantasy", "Un monde de dark fantasy médiévale", "JDR"),
        ("Pathfinder", "Un jeu de rôle d'heroic fantasy", "JDR"),
        ("Vampire: La Mascarade", "Un jeu gothique-punk où vous incarnez des vampires", "JDR"),
        ("Cyberpunk RED", "Un jeu de rôle futuriste dans un monde dystopique", "JDR"),
        ("Star Wars: Edge of the Empire", "Aventures dans l'univers de Star Wars", "JDR"),
        ("Chroniques Oubliées", "Un jeu de rôle médiéval-fantastique accessible", "JDR"),
        ("Shadowrun", "Mélange de cyberpunk et de fantasy avec magie et technologie", "JDR"),
        ("Blades in the Dark", "Un jeu de criminels dans une cité victorienne hantée", "JDR"),
        ("Tales from the Loop", "Aventures d'enfants dans les années 80 alternatives", "JDR"),
        ("Loup-Garou l'Apocalypse", "Incarnez des guerriers-loups défenseurs de la nature", "JDR"),
        ("Le Seigneur des Anneaux JdR", "Aventures dans la Terre du Milieu", "JDR"),
        ("Alien RPG", "Horreur et survie dans l'espace", "JDR"),
        ("Numenera", "Science-fantasy dans un futur très lointain", "JDR"),
        
        # Jeux de société
        ("Catan", "Jeu de société de stratégie et de développement", "JEU_DE_SOCIETE"),
        ("7 Wonders", "Jeu de cartes et de civilisations", "JEU_DE_SOCIETE"),
        ("Pandemic", "Jeu coopératif pour sauver l'humanité", "JEU_DE_SOCIETE"),
        ("Terraforming Mars", "Jeu de stratégie sur la colonisation de Mars", "JEU_DE_SOCIETE"),
        ("Carcassonne", "Jeu de tuiles et de territoire", "JEU_DE_SOCIETE"),
        ("Dixit", "Jeu d'imagination et d'interprétation d'images", "JEU_DE_SOCIETE"),
        ("Splendor", "Jeu de développement économique dans la Renaissance", "JEU_DE_SOCIETE"),
        ("Les Aventuriers du Rail", "Construction de routes ferroviaires", "JEU_DE_SOCIETE"),
        ("Azul", "Jeu de placement de tuiles aux motifs inspirés d'azulejos", "JEU_DE_SOCIETE"),
        ("Codenames", "Jeu d'association de mots en équipes", "JEU_DE_SOCIETE"),
        ("King of Tokyo", "Affrontement de monstres géants", "JEU_DE_SOCIETE"),
        ("Dominion", "Jeu de construction de deck médiéval", "JEU_DE_SOCIETE"),
        ("Scythe", "Conquête de territoires dans un monde uchronique", "JEU_DE_SOCIETE"),
        ("Root", "Lutte asymétrique pour le contrôle de la forêt", "JEU_DE_SOCIETE"),
        ("Wingspan", "Collection d'oiseaux et création d'habitat", "JEU_DE_SOCIETE")
    ]
    
    # Préparation des données de jeux avec les images
    jeux_data = []
    for nom_jeu, description, type_jeu in jeux_base:
        # Chercher si le jeu a une image associée dans notre dictionnaire
        image = None
        
        # Gérer le cas de Warhammer (Warhammer Fantasy dans la base, mais Warhammer dans le dictionnaire d'images)
        recherche = "Warhammer" if nom_jeu.startswith("Warhammer") else nom_jeu
        
        # Pour Vampire, le nom exact est "Vampire: La Mascarade"
        if recherche == "Vampire: La Mascarade":
            image = images_jeux.get(recherche)
        else:
            # Recherche partielle pour les autres jeux
            for cle in images_jeux:
                if cle in recherche:
                    image = images_jeux[cle]
                    break
        
        jeux_data.append((nom_jeu, description, type_jeu, image))
    
    jeux_sql = """
    INSERT INTO jeux (nom, description, type_jeu, image)
    VALUES (%s, %s, %s, %s)
    """
    
    cursor.executemany(jeux_sql, jeux_data)
    conn.commit()
    print(f"✅ {len(jeux_data)} jeux créés avec succès!")
    
    # --- OBTENTION DES IDS DES JEUX CRÉÉS ---
    cursor.execute("SELECT id, nom, image FROM jeux")
    jeux_info = cursor.fetchall()
    jeux_ids = [row[0] for row in jeux_info]
    
    # Créer un dictionnaire pour lier les ID des jeux à leurs images
    jeux_images_dict = {row[0]: row[2] for row in jeux_info}
    
        # --- GÉNÉRATION DES GENRES ---
    print("Génération des genres...")
    
    # Définition des genres pour les jeux de rôle et de société
    genres_list = [
        "Fantastique", 
        "Horreur", 
        "Science-fiction", 
        "Post-apocalyptique",
        "Médiéval", 
        "Contemporain", 
        "Steampunk", 
        "Cyberpunk", 
        "Heroic-Fantasy", 
        "Space Opera",
        "Dark Fantasy", 
        "Western", 
        "Historique", 
        "Espionnage", 
        "Enquête",
        "Coopératif", 
        "Stratégie", 
        "Gestion", 
        "Deck-building", 
        "Placement d'ouvriers"
    ]
    
    # Insérer les genres dans la base de données
    genres_data = [(genre,) for genre in genres_list]
    
    # Utiliser INSERT IGNORE pour éviter les erreurs en cas de duplicata
    genres_sql = """
    INSERT IGNORE INTO genres (nom)
    VALUES (%s)
    """
    
    cursor.executemany(genres_sql, genres_data)
    conn.commit()
    print(f"✅ Genres créés avec succès!")
    
    # --- GÉNÉRATION DES ASSOCIATIONS JEUX-GENRES ---
    print("Association des jeux aux genres...")
    
    # Récupération des ids de genres
    cursor.execute("SELECT id FROM genres")
    genres_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES ASSOCIATIONS JEUX-GENRES ---
    print("Association des jeux aux genres...")
    
    # Récupération des ids de genres
    cursor.execute("SELECT id FROM genres")
    genres_ids = [row[0] for row in cursor.fetchall()]
    
    jeux_genres_data = []
    for jeu_id in jeux_ids:
        # Chaque jeu aura 1 à 3 genres aléatoires
        nombre_genres = min(random.randint(1, 3), len(genres_ids))
        genres_choisis = random.sample(genres_ids, nombre_genres)
        
        for genre_id in genres_choisis:
            jeux_genres_data.append((jeu_id, genre_id))
    
    # Éliminer les doublons
    jeux_genres_data = list(set(jeux_genres_data))
    
    jeux_genres_sql = """
    INSERT INTO jeux_genres (id_jeu, id_genre)
    VALUES (%s, %s)
    """
    
    cursor.executemany(jeux_genres_sql, jeux_genres_data)
    conn.commit()
    print(f"✅ {len(jeux_genres_data)} associations jeux-genres créées!")
    
    # --- GÉNÉRATION DES LIEUX ---
    print("Génération des lieux...")
    
    lieux_data = [
        ("Maison des Associations", "123 Rue des Jeux, 45000 Orléans", "Salle principale de l'association", 47.902964, 1.909251),
        ("FSV", "456 Avenue du Gaming, 45100 Orléans", "Fédération des Sports Virtuels", 47.913427, 1.873779),
        ("Discord", "En ligne", "Serveur Discord de l'association", None, None),
        ("Bibliothèque Municipale", "789 Boulevard des Livres, 45000 Orléans", "Salle de jeux au sous-sol", 47.899631, 1.900110),
        ("Café Ludique", "10 Place du Plateau, 45000 Orléans", "Café avec jeux de société", 47.905783, 1.899988),
        ("Centre Culturel", "42 Avenue des Arts, 45000 Orléans", "Grande salle pour événements", 47.895631, 1.904233),
        ("Salle Polyvalente", "15 Rue des Rencontres, 45100 Orléans", "Espace modulable pour grands groupes", 47.920154, 1.887542),
        ("Roll20", "En ligne", "Plateforme de jeu de rôle virtuelle", None, None),
        ("Bar Le D20", "27 Rue du Hasard, 45000 Orléans", "Bar à thème jeux de société", 47.908743, 1.912564),
        ("Local Associatif", "8 Impasse des Joueurs, 45000 Orléans", "Petit local de l'association", 47.903126, 1.906873)
    ]
    
    lieux_sql = """
    INSERT INTO lieux (nom, adresse, description, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s)
    """
    
    cursor.executemany(lieux_sql, lieux_data)
    conn.commit()
    print(f"✅ {len(lieux_data)} lieux créés avec succès!")
    
    # --- OBTENTION DES IDS DES LIEUX CRÉÉS ---
    cursor.execute("SELECT id FROM lieux")
    lieux_ids = [row[0] for row in cursor.fetchall()]
    
    # --- OBTENTION DES IDS DES UTILISATEURS CRÉÉS ---
    cursor.execute("SELECT id FROM utilisateurs")
    utilisateurs_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES PARTIES ---
    print("Génération des parties...")
    
    types_partie = ["CAMPAGNE", "ONESHOT", "JEU_DE_SOCIETE", "EVENEMENT"]
    types_campagne = ["OUVERTE", "FERMEE"]
    
    parties_data = []
    for i in range(15):
        nom = f"Partie {i+1}: L'aventure fantastique"
        id_jeu = random.choice(jeux_ids)
        id_maitre_jeu = random.choice(utilisateurs_ids)
        type_partie = random.choice(types_partie)
        type_campagne = random.choice(types_campagne) if type_partie == "CAMPAGNE" else None
        description_courte = f"Une aventure palpitante vous attend dans cette partie de jeu n°{i+1}!"
        description = f"Description complète de la partie {i+1}. Beaucoup de détails sur l'univers et les règles."
        nombre_max_joueurs = random.randint(6,20) if type_campagne == "OUVERTE" else random.randint(3, 8)
        max_joueurs_session = min(nombre_max_joueurs, random.randint(2, 5))
        verrouille = random.random() > 0.8
        
        # Récupérer l'image du jeu associé pour l'utiliser comme image de la partie
        image = jeux_images_dict.get(id_jeu)
        texte_alt_image = f"Image pour la partie {nom}" if image else None
        
        parties_data.append((
            nom, id_jeu, id_maitre_jeu, type_partie, type_campagne,
            description_courte, description, nombre_max_joueurs,
            max_joueurs_session, verrouille, image, texte_alt_image
        ))
    
    parties_sql = """
    INSERT INTO parties (
        nom, id_jeu, id_maitre_jeu, type_partie, type_campagne,
        description_courte, description, nombre_max_joueurs,
        max_joueurs_session, verrouille, image, texte_alt_image
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(parties_sql, parties_data)
    conn.commit()
    print(f"✅ {len(parties_data)} parties créées avec succès!")
    
    # --- OBTENTION DES IDS DES PARTIES CRÉÉES ---
    cursor.execute("SELECT id FROM parties")
    parties_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES SESSIONS ---
    print("Génération des sessions...")

    sessions_data = []
    for partie_id in parties_ids:
        # Chaque partie aura 1 à 5 sessions
        for j in range(random.randint(1, 5)):
            id_partie = partie_id
            id_lieu = random.choice(lieux_ids)
            # Les sessions sont programmées dans le futur
            date_session = (datetime.now() + timedelta(days=random.randint(1, 60))).strftime('%Y-%m-%d')
            heure_debut = f"{random.randint(18, 20)}:00:00"
            heure_fin = f"{random.randint(21, 23)}:00:00"
            
            # Récupérer le maître de jeu de la partie
            cursor.execute("SELECT id_maitre_jeu FROM parties WHERE id = %s", (id_partie,))
            id_maitre_jeu = cursor.fetchone()[0]
            
            # Récupérer max_joueurs_session de la partie
            cursor.execute("SELECT max_joueurs_session FROM parties WHERE id = %s", (id_partie,))
            max_joueurs_session = cursor.fetchone()[0]
            
            # Le nombre max de joueurs est souvent le même que celui de la partie, mais peut varier
            nombre_max_joueurs = max_joueurs_session if random.random() > 0.3 else random.randint(2, max_joueurs_session)
            
            # Nous utilisons seulement nombre_max_joueurs dans la table sessions
            sessions_data.append((
                id_partie, id_lieu, date_session, heure_debut, heure_fin,
                id_maitre_jeu, nombre_max_joueurs
            ))

    sessions_sql = """
    INSERT INTO sessions (
        id_partie, id_lieu, date_session, heure_debut, heure_fin,
        id_maitre_jeu, nombre_max_joueurs
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    
    cursor.executemany(sessions_sql, sessions_data)
    conn.commit()
    print(f"✅ {len(sessions_data)} sessions créées avec succès!")
    
    # --- OBTENTION DES IDS DES SESSIONS CRÉÉES ---
    cursor.execute("SELECT id FROM sessions")
    sessions_ids = [row[0] for row in cursor.fetchall()]
    
    # --- GÉNÉRATION DES INSCRIPTIONS AUX SESSIONS ---
    print("Génération des inscriptions aux sessions...")
    
    # Dans la partie génération des inscriptions aux sessions
    joueurs_session_data = []
    for session_id in sessions_ids:
        # Récupérer le max_joueurs_session pour cette session
        cursor.execute("SELECT nombre_max_joueurs FROM sessions WHERE id = %s", (session_id,))
        max_joueurs = cursor.fetchone()[0]
        
        # Récupérer le maître de jeu de cette session pour ne pas l'inscrire comme joueur
        cursor.execute("SELECT id_maitre_jeu FROM sessions WHERE id = %s", (session_id,))
        id_maitre_jeu = cursor.fetchone()[0]
        
        # Vérifier les joueurs déjà inscrits à cette session
        cursor.execute("SELECT id_utilisateur FROM joueurs_session WHERE id_session = %s", (session_id,))
        joueurs_existants = [row[0] for row in cursor.fetchall()]
        
        # Chaque session aura un nombre aléatoire de joueurs inscrits
        nombre_joueurs = random.randint(0, max_joueurs)
        joueurs_potentiels = [u for u in utilisateurs_ids if u != id_maitre_jeu and u not in joueurs_existants]
        
        if joueurs_potentiels:
            joueurs_selectionnes = random.sample(joueurs_potentiels, min(nombre_joueurs, len(joueurs_potentiels)))
            
            for joueur_id in joueurs_selectionnes:
                date_inscription = datetime.now() - timedelta(days=random.randint(1, 30))
                joueurs_session_data.append((
                    session_id, joueur_id, date_inscription.strftime('%Y-%m-%d %H:%M:%S')
            ))
    
    joueurs_session_sql = """
    INSERT INTO joueurs_session (
        id_session, id_utilisateur, date_inscription
    ) VALUES (%s, %s, %s)
    """
    
    cursor.executemany(joueurs_session_sql, joueurs_session_data)
    conn.commit()
    print(f"✅ {len(joueurs_session_data)} inscriptions aux sessions créées!")
    
    # --- GÉNÉRATION DES MEMBRES DE PARTIES FERMÉES ---
    print("Génération des membres de parties fermées...")
    
    # Dans la partie génération des membres de parties fermées
    membres_partie_data = []
    for partie_id in parties_ids:
        # Vérifier si c'est une campagne fermée
        cursor.execute("SELECT type_partie, type_campagne FROM parties WHERE id = %s", (partie_id,))
        result = cursor.fetchone()
        type_partie, type_campagne = result
        
        if type_partie == "CAMPAGNE" and type_campagne == "FERMEE":
            # Récupérer le maître de jeu de cette partie
            cursor.execute("SELECT id_maitre_jeu FROM parties WHERE id = %s", (partie_id,))
            id_maitre_jeu = cursor.fetchone()[0]
            
            # Vérifier les membres déjà inscrits à cette partie
            cursor.execute("SELECT id_utilisateur FROM membres_partie WHERE id_partie = %s", (partie_id,))
            membres_existants = [row[0] for row in cursor.fetchall()]
            
            # Récupérer le nombre max de joueurs pour cette partie
            cursor.execute("SELECT nombre_max_joueurs FROM parties WHERE id = %s", (partie_id,))
            max_joueurs = cursor.fetchone()[0]
            
            # Chaque partie fermée aura un nombre aléatoire de membres
            nombre_membres = random.randint(1, max_joueurs)
            membres_potentiels = [u for u in utilisateurs_ids if u != id_maitre_jeu and u not in membres_existants]
            
            if membres_potentiels:
                membres_selectionnes = random.sample(membres_potentiels, min(nombre_membres, len(membres_potentiels)))
                
                for membre_id in membres_selectionnes:
                    membres_partie_data.append((partie_id, membre_id))
                    
        if type_partie == "CAMPAGNE" and type_campagne == "OUVERTE":
            # Récupérer le maître de jeu de cette partie
            cursor.execute("SELECT id_maitre_jeu FROM parties WHERE id = %s", (partie_id,))
            id_maitre_jeu = cursor.fetchone()[0]
            
            # Vérifier les membres déjà inscrits à cette partie
            cursor.execute("SELECT id_utilisateur FROM membres_partie WHERE id_partie = %s", (partie_id,))
            membres_existants = [row[0] for row in cursor.fetchall()]
            
            # Récupérer le nombre max de joueurs pour cette partie
            cursor.execute("SELECT nombre_max_joueurs FROM parties WHERE id = %s", (partie_id,))
            max_joueurs = cursor.fetchone()[0]
            
            # Chaque partie fermée aura un nombre aléatoire de membres
            nombre_membres = random.randint(round(max_joueurs/2), max_joueurs)
            membres_potentiels = [u for u in utilisateurs_ids if u != id_maitre_jeu and u not in membres_existants]
            
            if membres_potentiels:
                membres_selectionnes = random.sample(membres_potentiels, min(nombre_membres, len(membres_potentiels)))
                
                for membre_id in membres_selectionnes:
                    membres_partie_data.append((partie_id, membre_id))
    
    membres_partie_sql = """
    INSERT INTO membres_partie (id_partie, id_utilisateur)
    VALUES (%s, %s)
    """
    
    cursor.executemany(membres_partie_sql, membres_partie_data)
    conn.commit()
    print(f"✅ {len(membres_partie_data)} membres de parties fermées créés!")
    
    print("\n✅ GÉNÉRATION DES DONNÉES TERMINÉE AVEC SUCCÈS! ✅")

except mysql.connector.Error as err:
    print(f"❌ Erreur: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("Connexion à la base de données fermée.")


Connexion à la base de données réussie!
✓ Table refresh_tokens vidée
✓ Table paiements_helloasso vidée
✓ Table notifications_helloasso vidée
✓ Table creneaux_utilisateur vidée
✓ Table horaires_lieu vidée
✓ Table joueurs_session vidée
✓ Table membres_partie vidée
✓ Table sessions vidée
✓ Table jeux_genres vidée
✓ Table evenements vidée
✓ Table parties vidée
✓ Table lieux vidée
✓ Table genres vidée
✓ Table jeux vidée
✓ Table periodes_association vidée
✓ Table utilisateurs vidée
✅ Toutes les tables ont été vidées avec succès!
Génération des utilisateurs...
✅ 20 utilisateurs créés avec succès!
Génération des jeux...
✅ 30 jeux créés avec succès!
Génération des genres...
✅ Genres créés avec succès!
Association des jeux aux genres...
Association des jeux aux genres...
✅ 59 associations jeux-genres créées!
Génération des lieux...
✅ 10 lieux créés avec succès!
Génération des parties...
✅ 15 parties créées avec succès!
Génération des sessions...
✅ 46 sessions créées avec succès!
Génération des i

# Script de génération de données de test pour la base de données LBDR

Ce notebook contient un script Python qui génère des données de test pour la base de données de l'application. Les données générées incluent :

- Des utilisateurs aléatoires
- Des jeux (JDR et jeux de société)
- Des genres associés aux jeux
- Des lieux pour les sessions
- Des parties (campagnes et one-shots)
- Des sessions liées aux parties
- Des inscriptions de joueurs aux sessions
- Des membres pour les campagnes fermées

Pour exécuter le script, assurez-vous que :
1. La base de données est accessible
2. Les tables ont été créées (le script ne crée pas les tables)
3. Le package `mysql-connector-python` est installé

Le script nettoie les tables existantes avant d'insérer de nouvelles données.